## Recuperación BDD

## Nombre: Joel Molina

## NRC: 8903

## Librerías

In [12]:
import pandas as pd 
import numpy as np 
import sqlalchemy
from sqlalchemy import create_engine

import pyodbc
import sqlite3
from datetime import datetime
import csv

## Leer archivo unconfirmed-transac

In [13]:
DataSet = pd.read_json('unconfirmed-transactions.json') 

In [14]:
DataSet

,hash,ver,vin_sz,vout_sz,size,weight,fee,relayed_by,lock_time,tx_index,double_spend,time,block_index,block_height,inputs,out,rbf
0,0da81116cefbe7b6ffe50f268aff60e54d128697321dca...,2,1,2,223,565,330,0.0.0.0,0,540025040595873,False,1645714695,NaN,NaN,"[{'sequence': 4294967295, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 78537, '...",NaN
1,65d6041692cac70e4808be6829d5ef55cd50726afff236...,2,1,2,224,566,330,0.0.0.0,0,8450672181646481,False,1645714694,NaN,NaN,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 140245, ...",NaN
2,57dfa973641f85df9a5a2ba793b97fa2abbd2d7c1236ff...,2,1,3,255,693,701,0.0.0.0,724755,6349567405916472,False,1645714694,NaN,NaN,"[{'sequence': 4294967293, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 89710, '...",1.0
3,f1e506d14465d58c7480910f0fe7f46d783d56e553ef24...,2,1,3,256,694,351,0.0.0.0,724755,5272799869882357,False,1645714694,NaN,NaN,"[{'sequence': 4294967293, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 40680, '...",1.0
4,ef9feb3078312177367868ed6624b33049915692546e50...,1,1,2,223,565,609,0.0.0.0,0,4419740363441975,False,1645714694,NaN,NaN,"[{'sequence': 4294967295, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 235797, ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,921a4c01677f7b92323dcd21b46c99f2cc77650e8dc939...,1,1,2,247,658,328,0.0.0.0,0,521671263832471,False,1645714671,NaN,NaN,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 280000, ...",NaN
96,8630fd9cdeae84d0b4f9975b03cc36333b2a83f50acad8...,2,2,1,341,713,783,0.0.0.0,0,8166712539021134,False,1645714670,NaN,NaN,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 835945, ...",NaN
97,2709f8e0aa352d43623810ebcd1be66bd6e78f8c8f49b3...,1,1,1,192,438,330,0.0.0.0,0,6651557556083685,False,1645714670,NaN,NaN,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 773143, ...",NaN
98,015286bdab4e4a5961b3864dce77533b77e5b3d3a849d8...,1,1,1,151,397,371,0.0.0.0,0,4478559112354972,False,1645714670,NaN,NaN,"[{'sequence': 4294967295, 'witness': '01415854...","[{'type': 0, 'spent': False, 'value': 47739, '...",NaN


## Conexión con SQL Server

In [15]:
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=LOCALHOST;DATABASE=BD_Transaccional_Building_Permits;UID='';PWD='' ')

## Contar los datos faltantes del archivo unconfirmed-transac

In [16]:
DatosFaltantes = DataSet.isnull().sum()
DatosFaltantes

hash              0
ver               0
vin_sz            0
vout_sz           0
size              0
weight            0
fee               0
relayed_by        0
lock_time         0
tx_index          0
double_spend      0
time              0
block_index     100
block_height    100
inputs            0
out               0
rbf              76
dtype: int64

## Eliminación de las columnas con mas del 50 % de datos nulos

In [17]:
descartarDatosF= DataSet.drop(['block_index', 'block_height'], axis=1)
descartarDatosF.head()

,hash,ver,vin_sz,vout_sz,size,weight,fee,relayed_by,lock_time,tx_index,double_spend,time,inputs,out,rbf
0,0da81116cefbe7b6ffe50f268aff60e54d128697321dca...,2,1,2,223,565,330,0.0.0.0,0,540025040595873,False,1645714695,"[{'sequence': 4294967295, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 78537, '...",NaN
1,65d6041692cac70e4808be6829d5ef55cd50726afff236...,2,1,2,224,566,330,0.0.0.0,0,8450672181646481,False,1645714694,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 140245, ...",NaN
2,57dfa973641f85df9a5a2ba793b97fa2abbd2d7c1236ff...,2,1,3,255,693,701,0.0.0.0,724755,6349567405916472,False,1645714694,"[{'sequence': 4294967293, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 89710, '...",1.0
3,f1e506d14465d58c7480910f0fe7f46d783d56e553ef24...,2,1,3,256,694,351,0.0.0.0,724755,5272799869882357,False,1645714694,"[{'sequence': 4294967293, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 40680, '...",1.0
4,ef9feb3078312177367868ed6624b33049915692546e50...,1,1,2,223,565,609,0.0.0.0,0,4419740363441975,False,1645714694,"[{'sequence': 4294967295, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 235797, ...",NaN


### Columnas del DataSet resultantes de la eliminación

In [18]:
print("Columnas resultantes: %d" % descartarDatosF.shape[1])

Columnas resultantes: 15


### Transformar los datos Nulos

In [19]:
DatosTransformados=descartarDatosF.fillna(method ='ffill')
DatosTransformados

,hash,ver,vin_sz,vout_sz,size,weight,fee,relayed_by,lock_time,tx_index,double_spend,time,inputs,out,rbf
0,0da81116cefbe7b6ffe50f268aff60e54d128697321dca...,2,1,2,223,565,330,0.0.0.0,0,540025040595873,False,1645714695,"[{'sequence': 4294967295, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 78537, '...",NaN
1,65d6041692cac70e4808be6829d5ef55cd50726afff236...,2,1,2,224,566,330,0.0.0.0,0,8450672181646481,False,1645714694,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 140245, ...",NaN
2,57dfa973641f85df9a5a2ba793b97fa2abbd2d7c1236ff...,2,1,3,255,693,701,0.0.0.0,724755,6349567405916472,False,1645714694,"[{'sequence': 4294967293, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 89710, '...",1.0
3,f1e506d14465d58c7480910f0fe7f46d783d56e553ef24...,2,1,3,256,694,351,0.0.0.0,724755,5272799869882357,False,1645714694,"[{'sequence': 4294967293, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 40680, '...",1.0
4,ef9feb3078312177367868ed6624b33049915692546e50...,1,1,2,223,565,609,0.0.0.0,0,4419740363441975,False,1645714694,"[{'sequence': 4294967295, 'witness': '02473044...","[{'type': 0, 'spent': False, 'value': 235797, ...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,921a4c01677f7b92323dcd21b46c99f2cc77650e8dc939...,1,1,2,247,658,328,0.0.0.0,0,521671263832471,False,1645714671,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 280000, ...",1.0
96,8630fd9cdeae84d0b4f9975b03cc36333b2a83f50acad8...,2,2,1,341,713,783,0.0.0.0,0,8166712539021134,False,1645714670,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 835945, ...",1.0
97,2709f8e0aa352d43623810ebcd1be66bd6e78f8c8f49b3...,1,1,1,192,438,330,0.0.0.0,0,6651557556083685,False,1645714670,"[{'sequence': 4294967295, 'witness': '02483045...","[{'type': 0, 'spent': False, 'value': 773143, ...",1.0
98,015286bdab4e4a5961b3864dce77533b77e5b3d3a849d8...,1,1,1,151,397,371,0.0.0.0,0,4478559112354972,False,1645714670,"[{'sequence': 4294967295, 'witness': '01415854...","[{'type': 0, 'spent': False, 'value': 47739, '...",1.0


## Carga de datos

In [21]:
conec = sqlalchemy.create_engine('mssql+pyodbc://JOELMOLINA/BD_Unconfirmed_Transactions?driver=SQL+Server+Native+Client+11.0')
try:
    DatosTransformados.to_sql(name='Unconfirmed_Transactions', con=conec, if_exists='append',index=False)
    print("CARGA EXITOSA")
    
except Exception as ex:
    print("Ocurrió un error: {}".format(ex))

Ocurrió un error: (pyodbc.ProgrammingError) ("A TVP's rows must be Sequence objects.", 'HY000')
[SQL: INSERT INTO [Unconfirmed_Transactions] (hash, ver, vin_sz, vout_sz, size, weight, fee, relayed_by, lock_time, tx_index, double_spend, time, inputs, out, rbf) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('0da81116cefbe7b6ffe50f268aff60e54d128697321dca49220e5db70533590f', 2, 1, 2, 223, 565, 330, '0.0.0.0', 0, 540025040595873, 0, 1645714695, [{'sequence': 4294967295, 'witness': '0247304402202d35dbb57a974bcfaedf8d1302fcffb339e5276c9dac0cbd26e57cc57e7fa574022016f6276876750b62272e877ebeca16cf ... (252 characters truncated) ...  540025040595873, 'n': 0}], 'tx_index': 1358168429833340, 'value': 81228, 'addr': 'bc1qee4v336fje5cw3xrtxwuxy9wn6ckjqsy0sy73a', 'n': 101, 'type': 0}}], [{'type': 0, 'spent': False, 'value': 78537, 'spending_outpoints': [], 'n': 0, 'tx_index': 540025040595873, 'script': 'a9142b184cbed65a9835bcb3d2a9e27 ... (133 characters truncated) ...  'n': 1, 